In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import os
import formula_reader

read the flow using formula_reader.standard_flow()


In [2]:
def cleaning_the_dataset(dataset_material):
    for ii in dataset_material.keys():
        if (ii.find('Unnamed') != -1):
            del dataset_material[ii]
        elif list(set(pd.isnull(dataset_material[ii].values)))[0]==True:
            del dataset_material[ii]
        else:
            pass

In [3]:
dataset_material = pd.read_excel('Data_Ghiringhelli.xlsx', 'Material Data')
dataset_atomic = pd.read_excel('Data_Ghiringhelli.xlsx',   'Atomic Data')
dataset_atomic.columns = ['Z', 'A', 'IP', 'EA', 'HOMO', 'LUMO', 'rs', 'rp', 'rd', '1D']
dataset_test = pd.read_csv('dataset_test.csv')
dataset_test_new = pd.read_csv('dataset_test_new.csv')

In [4]:
cleaning_the_dataset(dataset_atomic)
cleaning_the_dataset(dataset_material)

In [5]:
column_name = formula_reader.columns_name(dataset_atomic_columns=dataset_atomic.keys()[2:-1], 
                           num_of_element=['A','B'])

In [6]:
equation_list = ['(rp_B**3 - exp(rs_B))/((rp_A**2))', 
                 '(sqrt(fabs(EA_A))+sqrt(fabs(IP_B)))/((rp_A**2))']

In [7]:
df_form = pd.read_excel('Final wrap up.xlsx', sheet_name='ZincBlend 1D')

all_num,all_equations,all_avg_mse = [],[],[]
for str_lst in df_form['OAD_gen_1 RMSE']:
    try :
        lindex = str_lst.index('(')
        rindex = str_lst.rindex(')')
        equation=str_lst[lindex:rindex+1]
        all_equations.append(equation)
    except:
        pass

In [8]:
new_equation_list = []
old_rmse, old_r2, old_cls   =[],[],[]
new_rmse, new_r2, new_cls   =[],[],[]
coeff_list                  =[]
min_slops, min_intercepts   =[],[]
r2_scores, cls_acss         =[],[]
regression = 'On'
rounding   = 12
target_property = 'r2_score' # 'RMSE' or 'classification_accuracy'

actute_species = ['RS' if i < 0 else 'ZB' for i in dataset_test.DE]
def target_func(pred, actu):
    rmse = np.round(np.sqrt(mean_squared_error(pred, actu)),rounding)
    r2 = r2_score(pred, dataset_test.DE)
    pred_species = ['RS' if i < 0 else 'ZB' for i in pred]
    quant_res    = ([i for i, j in zip(actute_species, pred_species) if i == j])
    cls_acs      = (len(quant_res)*100/82)
    return rmse, r2, cls_acs
        
for equation in all_equations[:10]:#+all_equations[4:7]+all_equations[8:9]:
    slops, intercepts = [],[]
    equation = equation.replace('HOMOKS','HOMO')
    equation = equation.replace('LUMOKS','LUMO')
    eq_obj = formula_reader.equation_reader(equation_string=equation, 
                                            dataset=dataset_test)
    feature_array = np.array(eq_obj.run_equation_reader())
    feature_array = np.reshape(feature_array,(-1,1))
    a,b,c = eq_obj.get_individual_element()
    formula = eq_obj.get_formula()
    new_equation_list.append(formula)
    actu = dataset_test['DE'].values

    regressor = LinearRegression()
    regressor.fit(feature_array, actu)
    
    pred = regressor.predict(np.array(feature_array))
    rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
    old_rmse.append(target_func(pred=pred, actu=actu)[0])
    old_r2.append(target_func(pred=pred, actu=actu)[1])
    old_cls.append(target_func(pred=pred, actu=actu)[2])
    print('rmse: ',old_rmse[-1], 'r2 score: ', old_r2[-1], 'class accuracy: ',old_cls[-1])
    coeff = regressor.coef_
    intercept = regressor.intercept_
    
    temp_rmse, value_lst, temp_r2, temp_cls_acs = [],[],[],[]
    for ii in np.arange(-1.1,1.1,0.01):
        for jj in np.arange(-1.1,1.1,0.01):
                if np.round(ii,4)==0. or np.round(jj,4)==0.:
                    pass
                else:
                    x = (np.round(ii,4)*a + np.round(jj,4)*b)/(c)
                    if regression == 'On':
                        regressor = LinearRegression()
                        regressor.fit(np.array(x).reshape(-1,1), dataset_test.DE.values)
                        pred = regressor.predict(np.array(x).reshape(-1,1))
                        rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                        temp_rmse.append(rmse)
                        temp_r2.append(r2)
                        temp_cls_acs.append(cls_acs)
                        value_lst.append([np.round(ii,4), np.round(jj,4)])
                        slops.append(regressor.coef_)
                        intercepts.append(regressor.intercept_)
                    else:
                        pred = (coeff*x)+intercept
                        rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                        temp_rmse.append(rmse)
                        temp_r2.append(r2)
                        temp_cls_acs.append(cls_acs)
                        value_lst.append([np.round(ii,4), np.round(jj,4)])
    
    if target_property == 'RMSE':
        args = np.argmin(temp_rmse)
    
    elif target_property == 'classification_accuracy':
        args = np.argmax(temp_cls_acs)
        
    elif target_property == 'r2_score':
        args = np.argmax(temp_r2)
    
    print(value_lst[args], temp_rmse[args], temp_r2[args], temp_cls_acs[args])
    diff=  [0.1,0.01,0.001,0.0001]
    interval = [0.01,0.001,0.0001,0.00001]
    for ll in range(len(diff)):
        range_lst = value_lst[args]
        for ii in (np.arange(range_lst[0]-diff[ll],
                             range_lst[0]+diff[ll],interval[ll])):
            for jj in (np.arange(range_lst[1]-diff[ll], 
                                 range_lst[1]+diff[ll],interval[ll])):
                    if np.round(ii,4)==0. or np.round(jj,4)==0.:
                        pass
                    else:
                        x = (np.round(ii,4)*a + np.round(jj,4)*b)/(c)
                        if regression == 'On':
                            regressor = LinearRegression()
                            regressor.fit(np.array(x).reshape(-1,1), dataset_test.DE.values)
                            pred = regressor.predict(np.array(x).reshape(-1,1))
                            rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                            temp_rmse.append(rmse)
                            temp_r2.append(r2)
                            temp_cls_acs.append(cls_acs)
                            value_lst.append([np.round(ii,4), np.round(jj,4)])
                            slops.append(regressor.coef_)
                            intercepts.append(regressor.intercept_)
                        else:                            
                            pred = (coeff*x)+intercept
                            rmse, r2, cls_acs = target_func(pred=pred, actu=actu)
                            temp_rmse.append(rmse)
                            temp_r2.append(r2)
                            temp_cls_acs.append(cls_acs)
                            value_lst.append([np.round(ii,4), np.round(jj,4)])
        print(value_lst[args], temp_rmse[args], temp_r2[args], temp_cls_acs[args])    
    
    print('------------------------------------',len(temp_rmse))
    
    new_rmse.append(temp_rmse[args])
    new_r2.append(temp_r2[args])
    new_cls.append(temp_cls_acs[args])
    coeff_list.append(value_lst[args])
    min_slops.append(slops[args])
    min_intercepts.append(intercepts[args])

origional eqution:  ((EA_B - IP_B)/rp_A**2)
operation list   :  [1, 1, 2, 1]
sign list        :  ['+', '-', '+', '+']
feature list     :  ['EA_B', 'IP_B', 'rp_A']
rmse:  0.142252905887 r2 score:  0.886078002051067 class accuracy:  89.02439024390245
[-0.78, 0.94] 0.142067893489 0.8864077794971026 89.02439024390245
[-0.78, 0.94] 0.142067893489 0.8864077794971026 89.02439024390245
[-0.78, 0.94] 0.142067893489 0.8864077794971026 89.02439024390245
[-0.78, 0.94] 0.142067893489 0.8864077794971026 89.02439024390245
[-0.78, 0.94] 0.142067893489 0.8864077794971026 89.02439024390245
------------------------------------ 49663
origional eqution:  (rp_B**3 - exp(rs_B))/((rp_A**2))
operation list   :  [3, 'exp', 2, 1]
sign list        :  ['+', '-', '+', '+']
feature list     :  ['rp_B', 'rs_B', 'rp_A']
rmse:  0.119258983423 r2 score:  0.9225522234215511 class accuracy:  91.46341463414635
[-1.0, 1.06] 0.117948656734 0.9243727490813285 91.46341463414635
[-1.0, 1.06] 0.117948656734 0.9243727490813285 91

In [9]:
df = pd.DataFrame()
df['Equations'] = new_equation_list
df['old_rmse_for_full_dataset']=old_rmse
df['new_rmse']=new_rmse
df['old_r2'] = old_r2
df['new_r2'] = new_r2
df['old_class_accuracy']= old_cls
df['new_class_accuracy']= new_cls
df['individual_coeff'] = coeff_list
df['slops'] = [round(num[0], 3) for num in min_slops]
df['intecepts'] = min_intercepts
#df['classification accuracy'] = temp

In [10]:
#df.to_excel('optimization_using_r2_score.xlsx')

In [11]:
test_func = 0.633*(-0.85*dataset_test['rp_B'].values**2+1.09*np.exp(dataset_test['rs_B'].values))/dataset_test['rp_A'].values**2  -0.341319

In [12]:
pred_species = ['RS' if i < 0 else 'ZB' for i in test_func]
quant_res    = ([i for i, j in zip(actute_species, pred_species) if i == j])

In [13]:
all_equations[10:21]

['(sqrt(fabs(EA_A)) + sqrt(fabs(IP_B)))/((rp_A**2))',
 '(sqrt(fabs(EA_A)) + sqrt(fabs(IP_B)))/((rp_A**2))',
 '(sqrt(fabs(IP_B)) + exp(EA_B))/((rp_A**2))',
 '(sqrt(fabs(HOMOKS_B)) + sqrt(fabs(LUMOKS_B)))/((rp_A**2))',
 '(sqrt(fabs(EA_B)) - sqrt(fabs(IP_B)))/((rp_A**2))',
 '(sqrt(fabs(IP_B)) + exp(IP_A))/((rp_A**2))',
 '(sqrt(fabs(IP_B)) - exp(IP_B))/((rp_A**2))',
 '(sqrt(fabs(IP_B)) + exp(IP_B))/((rp_A**2))',
 '(sqrt(fabs(IP_B)) - exp(IP_A))/((rp_A**2))',
 '(IP_A + IP_B)/((rp_A**2))',
 '(IP_B - sqrt(fabs(IP_A)))/((rp_A**2))']

In [14]:
df

,Equations,old_rmse_for_full_dataset,new_rmse,old_r2,new_r2,old_class_accuracy,new_class_accuracy,individual_coeff,slops,intecepts
0,((EA_B - IP_B)/rp_A**2),0.142253,0.142068,0.886078,0.886408,89.024390,89.024390,"[-0.78, 0.94]",-0.122,-0.350042
1,(rp_B**3 - exp(rs_B))/((rp_A**2)),0.119259,0.117949,0.922552,0.924373,91.463415,91.463415,"[-1.0, 1.06]",0.697,-0.325150
2,(rp_B - exp(rp_A))/((rp_A**3)),0.125963,0.125963,0.912819,0.912820,90.243902,90.243902,"[-1.1, 1.09]",0.662,-0.666403
3,(rp_B**3 - exp(rs_B))/((rs_A**3)),0.126580,0.125618,0.911888,0.913337,86.585366,89.024390,"[-0.74, 0.78]",0.631,-0.299895
4,(sqrt(fabs(rp_B)) - exp(rp_A))/((rp_A**3)),0.127083,0.114328,0.911125,0.929267,91.463415,90.243902,"[-1.04, 0.78]",1.458,-0.972615
5,(rs_B**2 - exp(rp_A))/((rp_A**3)),0.129892,0.127906,0.906781,0.909865,91.463415,92.682927,"[-1.01, 0.85]",0.746,-0.606334
6,(sqrt(fabs(rd_B)) - exp(rp_A))/((rp_A**3)),0.129955,0.129465,0.906683,0.907450,93.902439,93.902439,"[-0.63, 0.61]",2.665,-1.332577
7,(rs_B - exp(rp_A))/((rp_A**3)),0.130404,0.118721,0.905976,0.923303,91.463415,90.243902,"[-1.03, 0.73]",1.277,-0.826923
8,(rp_B**2 - exp(rs_B))/((rp_A**2)),0.130407,0.118675,0.905971,0.923367,95.121951,92.682927,"[-0.74, 0.61]",1.423,-0.322918
9,(sqrt(fabs(IP_B)) + sqrt(fabs(EA_A)))/((rp_A**2)),0.130913,0.130402,0.905171,0.905979,90.243902,91.463415,"[-0.86, -0.63]",-0.348,-0.386341
